<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# default_exp models.base

In [ ]:
# export
from torch import nn


class TTSModel(nn.Module):
    def infer(self):
        raise NotImplemented

    def forward(self):
        raise NotImplemented

    def from_pretrained(self):
        raise NotImplemented

    def to_checkpoint(self):
        return dict(model=self.state_dict())

    @classmethod
    def create(cls, name, opts, folders, all_speakers=True):
        pass


#         model_cls = cls.get_class(name)
#         folders = pd.read_csv(folders)
#         for folder in folders:


#         return model_cls(opts)